In [7]:
!python run.py --fontnum 2 --gpu 0

Using TensorFlow backend.
W0928 13:14:14.980363 140180100904768 deprecation_wrapper.py:119] From /home/viplab/anaconda3/envs/hyperLine/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0928 13:14:14.994730 140180100904768 deprecation_wrapper.py:119] From /home/viplab/anaconda3/envs/hyperLine/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0928 13:14:15.019931 140180100904768 deprecation_wrapper.py:119] From /home/viplab/anaconda3/envs/hyperLine/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0928 13:14:15.020061 140180100904768 deprecation_wrapper.py:119] From /home/viplab/anaconda3/envs/hyperLine/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174